# GPX Writer - GPS Exchange Format

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://github.com/Logiqx/gps-wizard) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

In [1]:
import os
import sys
import time

from lxml import etree
from datetime import datetime

import unittest

from base_writer import BaseWriter

## Main Class

In [2]:
class GpxWriter(BaseWriter):
    '''GPX file - GPS Exchange Format'''

    def __init__(self, filename, tracks):
        '''Basic init just records the filename'''

        super().__init__(filename, tracks)


    def prepare(self, tracks):
        '''Prepare GPX prior to being saved'''

        xsi = 'http://www.w3.org/2001/XMLSchema-instance'
        gpx = etree.Element(
            'gpx', 
             {etree.QName(xsi, 'schemaLocation'): 'http://www.topografix.com/GPX/1/0 http://www.topografix.com/GPX/1/0/gpx.xsd'},
             creator='GPS Wizard', 
             version='1.0', 
             nsmap={None: 'http://www.topografix.com/GPX/1/0', 'xsi': xsi})

        for track in tracks:
            formats = self.getFormats(track)

            trk = etree.SubElement(gpx, 'trk')

            if track.name:
                name = etree.SubElement(trk, 'name')
                name.text = track.name

            trkseg = etree.SubElement(trk, 'trkseg')

            for i in range(track.numPoints):
                trkpt = etree.SubElement(trkseg, 'trkpt')

                for attrib in ['lat', 'lon']:
                    trkpt.attrib[attrib] = formats[attrib].format(track.data[attrib][i])

                for element, abbr in [('ele', 'ele'),
                                      ('time', 'ts'),
                                      ('course', 'cog'),
                                      ('speed', 'sog'),
                                      ('sat', 'sat'),
                                      ('hdop', 'hdop')]:
                    if abbr in track.data:
                        subElement = etree.SubElement(trkpt, element)
                        if element == 'time':
                            subElement.text = datetime.fromtimestamp(track.data[abbr][i]).isoformat() + 'Z'
                        elif abbr in formats:
                            subElement.text = formats[abbr].format(track.data[abbr][i])
                        else:
                            subElement.text = track.data[abbr][i]

            self.buffer = etree.tostring(gpx, pretty_print=True, xml_declaration=False, encoding='UTF-8',
                                         doctype='<?xml version="1.0" encoding="UTF-8"?>')


    def save(self):
        '''Save GPX to disk'''
        
        with open(self.filename, 'wb') as f:
            f.write(self.buffer)

## Unit Tests

In [3]:
class TestGpxWriter(unittest.TestCase):
    '''Class to test the GPX writer'''

    def testGpxWriter(self):
        '''Test the GPX writer'''

        filename = os.path.join(projdir, 'sessions', 'unittest.gpx')

        gpxWriter = GpxWriter(filename, sbnReader.tracks)

        gpxWriter.save()
        
        os.unlink(filename)

In [4]:
if __name__ == '__main__':
    for path in ['python', '.', '..']:
        readersPath = os.path.join(path, 'core')
        if readersPath not in sys.path:
            sys.path.extend([readersPath])

    from file_reader import getFileReader

    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    sbnFilename = os.path.join(projdir, 'sessions', '20071227', 'MIKE_G_1003053_20071227_165512_DLG.SBP')
    sbnReader = getFileReader(sbnFilename)

    pc1 = time.perf_counter()
    sbnReader.load()
    pc2 = time.perf_counter()

    print("\nTest file loaded in %0.2f seconds" % (pc2 - pc1))


Test file loaded in 0.01 seconds


In [5]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

.
----------------------------------------------------------------------
Ran 1 test in 0.061s

OK
